In [1]:
import pandas as pd
import numpy as np
import os
import sys
import re

In [2]:
filepath = "/Users/surajshashidhar/git/fiction/milestone4/"
similarity_file = "simialrity_milestone4_english.csv"
emotion_vector_file = "feature3_vectors_milestone4_english.csv"
old_feature_file = "full_chunk_vectors_2606.csv"

In [5]:
sim_df = pd.read_csv(filepath+similarity_file, header = 0)
emo_df = pd.read_csv(filepath+emotion_vector_file, header = 0)
old_feature_df = pd.read_csv(filepath+old_feature_file, header = 0)

In [4]:
sim_df.head(10)

,similarity,book_id
0,0.875707,pg36556
1,0.869395,pg105
2,0.837580,pg4084
3,0.832758,pg35297
4,0.829437,pg11581
5,0.823205,pg29366
6,0.821948,pg547
7,0.819861,pg41332
8,0.816052,pg31138
9,0.813740,pg14420


In [6]:
old_feature_df.head()

,bookId-chunkNo,F0,F1,F2,F3,F4,F5,F6,F7,F8,...,0,1,2,3,4,5,6,7,8,9
0,pg13312-9,0.0208,0.3259,0.1080,0.0859,0.0215,0.0248,0.0292,0.0607,0.0544,...,-0.027213,-0.045306,-0.14997,0.114716,0.005374,0.017917,-0.025134,-0.102872,0.074462,0.030568
1,pg13312-8,0.0228,0.3223,0.1593,0.0885,0.0201,0.0249,0.0290,0.0582,0.0574,...,-0.027213,-0.045306,-0.14997,0.114716,0.005374,0.017917,-0.025134,-0.102872,0.074462,0.030568
2,pg13312-7,0.0271,0.2717,0.1345,0.0869,0.0202,0.0235,0.0260,0.0560,0.0525,...,-0.027213,-0.045306,-0.14997,0.114716,0.005374,0.017917,-0.025134,-0.102872,0.074462,0.030568
3,pg13312-6,0.0216,0.3732,0.0655,0.0759,0.0203,0.0306,0.0297,0.0533,0.0494,...,-0.027213,-0.045306,-0.14997,0.114716,0.005374,0.017917,-0.025134,-0.102872,0.074462,0.030568
4,pg13312-5,0.0228,0.3466,0.1058,0.0834,0.0225,0.0277,0.0261,0.0592,0.0517,...,-0.027213,-0.045306,-0.14997,0.114716,0.005374,0.017917,-0.025134,-0.102872,0.074462,0.030568


In [7]:
emo_df.head()

,start_anger,start_anticipation,start_disgust,start_fear,start_joy,start_sadness,start_surprise,start_trust,end_anger,end_anticipation,end_disgust,end_fear,end_joy,end_sadness,end_surprise,end_trust,bname,bid
0,1.036837,1.835221,0.521992,1.029118,1.389433,1.012386,0.962948,2.212065,0.772125,1.705616,0.564444,1.239443,1.218821,1.173408,1.016169,2.309974,Out of the Ashes,pg13273
1,1.202225,1.547794,0.785077,0.953890,1.140352,1.278852,0.674192,2.417618,1.389464,1.834950,0.603804,1.726106,0.594164,1.017907,0.860994,1.972610,William Tell Told Again,pg7298
2,0.792108,1.579833,0.562231,0.925394,1.878342,1.253864,1.004684,2.003545,0.837069,1.661124,0.572815,1.002008,1.542001,1.436395,1.102630,1.845960,Miss Hildreth: A Novel- Volume 1,pg40431
3,0.639720,1.486011,0.560789,1.023485,1.029786,1.058510,0.841580,3.360120,1.094647,1.498997,0.828350,1.296479,1.069322,1.117798,0.729444,2.364963,"Tales of the Wonder Club, Volume II",pg34817
4,0.604783,1.702867,0.534271,1.136983,1.169181,1.132258,0.876161,2.843496,0.571445,2.249504,0.494156,0.978018,1.359219,1.371692,0.885597,2.090369,Bill Nye's Cordwood,pg33391


In [8]:
tmp_df = old_feature_df["bookId-chunkNo"].str.split("-", n = 1, expand = True) 

In [9]:
tmp_df.head()

,0,1
0,pg13312,9
1,pg13312,8
2,pg13312,7
3,pg13312,6
4,pg13312,5


In [10]:
old_feature_df["bookid"] = tmp_df[0]
old_feature_df["chunkid"] = tmp_df[1]

In [11]:
old_feature_df.head()

,bookId-chunkNo,F0,F1,F2,F3,F4,F5,F6,F7,F8,...,2,3,4,5,6,7,8,9,bookid,chunkid
0,pg13312-9,0.0208,0.3259,0.1080,0.0859,0.0215,0.0248,0.0292,0.0607,0.0544,...,-0.14997,0.114716,0.005374,0.017917,-0.025134,-0.102872,0.074462,0.030568,pg13312,9
1,pg13312-8,0.0228,0.3223,0.1593,0.0885,0.0201,0.0249,0.0290,0.0582,0.0574,...,-0.14997,0.114716,0.005374,0.017917,-0.025134,-0.102872,0.074462,0.030568,pg13312,8
2,pg13312-7,0.0271,0.2717,0.1345,0.0869,0.0202,0.0235,0.0260,0.0560,0.0525,...,-0.14997,0.114716,0.005374,0.017917,-0.025134,-0.102872,0.074462,0.030568,pg13312,7
3,pg13312-6,0.0216,0.3732,0.0655,0.0759,0.0203,0.0306,0.0297,0.0533,0.0494,...,-0.14997,0.114716,0.005374,0.017917,-0.025134,-0.102872,0.074462,0.030568,pg13312,6
4,pg13312-5,0.0228,0.3466,0.1058,0.0834,0.0225,0.0277,0.0261,0.0592,0.0517,...,-0.14997,0.114716,0.005374,0.017917,-0.025134,-0.102872,0.074462,0.030568,pg13312,5


In [12]:
out_df1 = pd.merge(left = old_feature_df, right = sim_df, how = "inner", left_on = ["bookid"], right_on = "book_id")

In [13]:
out_df1.head()

,bookId-chunkNo,F0,F1,F2,F3,F4,F5,F6,F7,F8,...,4,5,6,7,8,9,bookid,chunkid,similarity,book_id
0,pg13312-9,0.0208,0.3259,0.1080,0.0859,0.0215,0.0248,0.0292,0.0607,0.0544,...,0.005374,0.017917,-0.025134,-0.102872,0.074462,0.030568,pg13312,9,0.727662,pg13312
1,pg13312-8,0.0228,0.3223,0.1593,0.0885,0.0201,0.0249,0.0290,0.0582,0.0574,...,0.005374,0.017917,-0.025134,-0.102872,0.074462,0.030568,pg13312,8,0.727662,pg13312
2,pg13312-7,0.0271,0.2717,0.1345,0.0869,0.0202,0.0235,0.0260,0.0560,0.0525,...,0.005374,0.017917,-0.025134,-0.102872,0.074462,0.030568,pg13312,7,0.727662,pg13312
3,pg13312-6,0.0216,0.3732,0.0655,0.0759,0.0203,0.0306,0.0297,0.0533,0.0494,...,0.005374,0.017917,-0.025134,-0.102872,0.074462,0.030568,pg13312,6,0.727662,pg13312
4,pg13312-5,0.0228,0.3466,0.1058,0.0834,0.0225,0.0277,0.0261,0.0592,0.0517,...,0.005374,0.017917,-0.025134,-0.102872,0.074462,0.030568,pg13312,5,0.727662,pg13312


In [14]:
out_df2 = pd.merge(left = out_df1, right = emo_df, how = "inner", left_on = ["bookid"], right_on = "bid")

In [15]:
out_df2.head()

,bookId-chunkNo,F0,F1,F2,F3,F4,F5,F6,F7,F8,...,end_anger,end_anticipation,end_disgust,end_fear,end_joy,end_sadness,end_surprise,end_trust,bname,bid
0,pg13312-9,0.0208,0.3259,0.1080,0.0859,0.0215,0.0248,0.0292,0.0607,0.0544,...,0.928691,1.855235,0.503938,1.293311,1.218693,1.11979,0.874432,2.205911,The Light That Lures,pg13312
1,pg13312-8,0.0228,0.3223,0.1593,0.0885,0.0201,0.0249,0.0290,0.0582,0.0574,...,0.928691,1.855235,0.503938,1.293311,1.218693,1.11979,0.874432,2.205911,The Light That Lures,pg13312
2,pg13312-7,0.0271,0.2717,0.1345,0.0869,0.0202,0.0235,0.0260,0.0560,0.0525,...,0.928691,1.855235,0.503938,1.293311,1.218693,1.11979,0.874432,2.205911,The Light That Lures,pg13312
3,pg13312-6,0.0216,0.3732,0.0655,0.0759,0.0203,0.0306,0.0297,0.0533,0.0494,...,0.928691,1.855235,0.503938,1.293311,1.218693,1.11979,0.874432,2.205911,The Light That Lures,pg13312
4,pg13312-5,0.0228,0.3466,0.1058,0.0834,0.0225,0.0277,0.0261,0.0592,0.0517,...,0.928691,1.855235,0.503938,1.293311,1.218693,1.11979,0.874432,2.205911,The Light That Lures,pg13312


In [16]:
out_df2.describe

<bound method NDFrame.describe of       bookId-chunkNo      F0      F1      F2      F3      F4      F5      F6  \
0          pg13312-9  0.0208  0.3259  0.1080  0.0859  0.0215  0.0248  0.0292   
1          pg13312-8  0.0228  0.3223  0.1593  0.0885  0.0201  0.0249  0.0290   
2          pg13312-7  0.0271  0.2717  0.1345  0.0869  0.0202  0.0235  0.0260   
3          pg13312-6  0.0216  0.3732  0.0655  0.0759  0.0203  0.0306  0.0297   
4          pg13312-5  0.0228  0.3466  0.1058  0.0834  0.0225  0.0277  0.0261   
...              ...     ...     ...     ...     ...     ...     ...     ...   
13583       pg9747-7  0.0180  0.3194  0.1534  0.0761  0.0197  0.0250  0.0376   
13584       pg9747-9  0.0304  0.2923  0.0855  0.0992  0.0194  0.0154  0.0303   
13585       pg1210-1  0.0132  0.3019  0.1154  0.0644  0.0214  0.0273  0.0375   
13586       pg1210-2  0.0116  0.3402  0.1195  0.0705  0.0224  0.0221  0.0420   
13587       pg1210-3  0.0094  0.2443  0.0534  0.0389  0.0137  0.0233  0.0405   

     

In [17]:
out_df2.columns

Index(['bookId-chunkNo', 'F0', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8',
       'F9', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18',
       'F19', 'F20', 'F21', 'F22', 'F23', '0', '1', '2', '3', '4', '5', '6',
       '7', '8', '9', 'bookid', 'chunkid', 'similarity', 'book_id',
       'start_anger', 'start_anticipation', 'start_disgust', 'start_fear',
       'start_joy', 'start_sadness', 'start_surprise', 'start_trust',
       'end_anger', 'end_anticipation', 'end_disgust', 'end_fear', 'end_joy',
       'end_sadness', 'end_surprise', 'end_trust', 'bname', 'bid'],
      dtype='object')

In [18]:
out_df2.drop(columns =["bookid", "chunkid", "book_id", "bname", "bid"], inplace = True)

In [19]:
out_df2.columns


Index(['bookId-chunkNo', 'F0', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8',
       'F9', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18',
       'F19', 'F20', 'F21', 'F22', 'F23', '0', '1', '2', '3', '4', '5', '6',
       '7', '8', '9', 'similarity', 'start_anger', 'start_anticipation',
       'start_disgust', 'start_fear', 'start_joy', 'start_sadness',
       'start_surprise', 'start_trust', 'end_anger', 'end_anticipation',
       'end_disgust', 'end_fear', 'end_joy', 'end_sadness', 'end_surprise',
       'end_trust'],
      dtype='object')

In [20]:
out_df2.to_csv(filepath+"Features_extracted_milestone4.csv", index=False)